# IPF Coloring

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from materialite import (
    Material,
    Orientation,
    Vector,
    get_ipf_colors,
    import_dream3d,
    add_ipf_colors_field,
)
from PIL import Image
import pyvista as pv
import matplotlib.pyplot as plt

Display IPF map legend

In [ ]:
ipf_legend = np.asarray(Image.open("ipf_example/ipf_map_legend_cubic.png"))
fig, ax = plt.subplots()
ax.imshow(ipf_legend)
ax.axis("off")

Simple example with three grains that have local $[100]$, $[110]$, and $[111]$ directions, respectively, aligned with the specimen $z$ direction

In [ ]:
euler_angles = [
    [0, 0, 0],
    [0, -np.pi / 4, 0],
    [0, np.arccos(1 / np.sqrt(3)), np.pi / 4],
]

grain_orientations = Orientation.from_euler_angles(euler_angles)
regional_field = {"grain": [0, 1, 2], "orientation": grain_orientations}
material = (
    Material([4, 4, 4])
    .create_voronoi(3, "grain", rng=np.random.default_rng(12345))
    .create_regional_fields(region_label="grain", regional_fields=regional_field)
    .run(add_ipf_colors_field)
)

In [ ]:
material.plot("ipf_color", kind="ipf_map")

Example with random orientations for a Voronoi tesselation

In [ ]:
material = (
    Material([128, 128, 128])
    .create_voronoi(500, "grain")
    .assign_random_orientations("grain")
    .run(add_ipf_colors_field)
)

In [ ]:
material.plot("ipf_color", kind="ipf_map")

Comparison with DREAM.3D-generated IPF map

In [ ]:
data_container = "DataContainers/SyntheticVolumeDataContainer"
material = import_dream3d(
    file="ipf_example/example_for_ipf_map.dream3d",
    simpl_geometry_path=f"{data_container}/_SIMPL_GEOMETRY",
    region_id_path=f"{data_container}/EBSD Scan Data/FeatureIds",
    region_field_paths=[
        f"{data_container}/Grain Data/EulerAngles",
    ],
    field_paths=[f"{data_container}/EBSD Scan Data/IPFColor"],
)
ipf_colors_dream3d = (
    material.extract(["i_p_f_color_1", "i_p_f_color_2", "i_p_f_color_3"]) / 255.0
)
eulers = material.extract(["euler_angles_1", "euler_angles_2", "euler_angles_3"])
orientations = Orientation.from_euler_angles(eulers)

material = material.create_fields(
    {
        "ipf_color_d3d": ipf_colors_dream3d.tolist(),
        "orientation": orientations,
        "grain": list(material.extract("feature_ids")),
    }
).run(add_ipf_colors_field)

Materialite IPF map

In [62]:
material.plot("ipf_color", kind="ipf_map")

DREAM.3D IPF map

In [63]:
material.plot("ipf_color_d3d", kind="ipf_map")

Compare with same grain morphology and Euler angles, but with an HCP lattice

In [ ]:
ipf_legend = np.asarray(Image.open("ipf_example/ipf_map_legend_hcp.png"))
fig, ax = plt.subplots()
ax.imshow(ipf_legend)
ax.axis("off")

In [ ]:
data_container = "DataContainers/SyntheticVolumeDataContainer"
material = import_dream3d(
    file="ipf_example/example_for_ipf_map_hcp.dream3d",
    simpl_geometry_path=f"{data_container}/_SIMPL_GEOMETRY",
    region_id_path=f"{data_container}/EBSD Scan Data/FeatureIds",
    region_field_paths=[
        f"{data_container}/Grain Data/EulerAngles",
    ],
    field_paths=[f"{data_container}/EBSD Scan Data/IPFColor"],
)
ipf_colors_dream3d = (
    material.extract(["i_p_f_color_1", "i_p_f_color_2", "i_p_f_color_3"]) / 255.0
)
eulers = material.extract(["euler_angles_1", "euler_angles_2", "euler_angles_3"])
orientations = Orientation.from_euler_angles(eulers)

material = material.create_fields(
    {
        "ipf_color_d3d": ipf_colors_dream3d.tolist(),
        "orientation": orientations,
        "grain": list(material.extract("feature_ids")),
    }
).run(add_ipf_colors_field, unit_cell="hcp")

Materialite IPF map

In [ ]:
material.plot("ipf_color", kind="ipf_map")

DREAM.3D IPF map

In [ ]:
material.plot("ipf_color_d3d", kind="ipf_map")